# Simple GRU network with pretrained vectors for initialization

In [1]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd
import tensorflow as tf
#from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, Permute, GRU, Conv1D, LSTM, Embedding, Dropout, Activation, CuDNNLSTM, CuDNNGRU, concatenate, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalAveragePooling1D, BatchNormalization, SpatialDropout1D, Dot
from keras.optimizers import Adam
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras_tqdm import TQDMNotebookCallback
import keras.backend as K
from keras.callbacks import LearningRateScheduler
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from functools import reduce
from keras.layers import Layer, PReLU, SpatialDropout1D
from keras import initializers
from sklearn.model_selection import cross_val_predict

from nltk.tokenize import word_tokenize, wordpunct_tokenize, TweetTokenizer, MWETokenizer, ToktokTokenizer
from nltk.corpus import stopwords

import unicodedata
from collections import Counter
import itertools

np.random.seed(786)

from Tokenizer import Tokenizer
from ZeroMaskedLayer import ZeroMaskedLayer
from AttentionLayer import AttentionLayer

/home/mohsin/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
path = '../input/'
utility_path = '../utility/'
comp = 'jigsaw-toxic-comment-classification-challenge/'
EMBEDDING_FILE=f'{utility_path}crawl-300d-2M.vec'
TRAIN_DATA_FILE=f'{path}train.csv'
TEST_DATA_FILE=f'{path}test.csv'

In [3]:
from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: %d - score: %.6f \n" % (epoch+1, score))

In [4]:
def unicodeToAscii(series):
    return series.apply(lambda s: unicodedata.normalize('NFKC', str(s)))


def multiple_replace(text, adict):
    rx = re.compile('|'.join(map(re.escape, adict)))

    def one_xlat(match):
        return adict[match.group(0)]

    return rx.sub(one_xlat, text)

STOP_WORDS = set(stopwords.words( 'english' ))
# Lowercase, trim, and remove non-letter characters
def normalizeString(series):
    series = unicodeToAscii(series)
    series = series.str.lower()
    series = series.str.replace(r"(\n){1,}", "")
    series = series.str.replace(r"\'", "")
    series = series.str.replace(r"\-", "")
    series = series.str.replace(r"[^0-9a-zA-Z.,!?]+", " ")

    return series


In [5]:
train = pd.read_csv(TRAIN_DATA_FILE)
test = pd.read_csv(TEST_DATA_FILE)

print(train.shape, test.shape)

list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values

#Get validation folds
train['target_str'] = reduce(lambda x,y: x+y, [train[col].astype(str) for col in list_classes])
train['target_str'] = train['target_str'].replace('110101', '000000').replace('110110','000000')
cvlist1 = list(StratifiedKFold(n_splits=10, random_state=786).split(train, train['target_str'].astype('category')))
cvlist2 = list(StratifiedShuffleSplit(n_splits=5, test_size=0.05, random_state=786).split(train, train['target_str'].astype('category')))

(159571, 8) (153164, 2)


/home/mohsin/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


In [6]:
for df in train, test:
    df["comment_text"] = normalizeString(df["comment_text"])

In [7]:
train.comment_text.sample(1).values[0]

'i will test alternate summaries for greater clarity.'

In [8]:
MAX_FEATURES = 200000
MAX_LEN = 250

tok = Tokenizer(max_features=MAX_FEATURES, max_len=MAX_LEN, tokenizer=wordpunct_tokenize)
X_train = tok.fit_transform(train["comment_text"].astype(str))
X_test = tok.transform(test["comment_text"].astype(str))

print(X_train.shape, X_test.shape)

(159571, 250) (153164, 250)


In [9]:
del train, test

In [10]:
EMBED_SIZE = 300
def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')

def initialize_embeddings(filename, tokenizer):
    embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(filename))

    word_index = tokenizer.vocab_idx
    nb_words = min(MAX_FEATURES, len(word_index))
    embedding_matrix = np.zeros((nb_words, EMBED_SIZE))
    for word, i in word_index.items():
        if i >= MAX_FEATURES: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [11]:
embedding_matrix = initialize_embeddings(EMBEDDING_FILE, tok)
print(embedding_matrix.shape)
print(np.mean(embedding_matrix), np.std(embedding_matrix))

(200000, 300)
0.002498661320857451 0.21826940261470665


In [12]:
from sklearn.base import BaseEstimator, ClassifierMixin
class GRUClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, gru_dim=150, dense_dim=256, batch_size=128, epochs=2, bidirectional=False, 
                 pool_type='all', initial_weights=None, optimizer='adam' ,verbose=1, out_dim=6, callbacks=None):
        
        self.gru_dim = gru_dim
        self.dense_dim = dense_dim
        self.batch_size = batch_size
        self.epochs= epochs
        self.bidirectional = bidirectional
        self.pool_type = pool_type
        self.initial_weights = initial_weights
        self.verbose = verbose
        self.callbacks = callbacks
        self.optimizer = optimizer
        self.out_dim = out_dim
    def _build_model(self):
        inp = Input(shape=(MAX_LEN,))
        emb = Embedding(MAX_FEATURES, 
                        EMBED_SIZE,
                        weights=[self.initial_weights],
                        mask_zero=True,
                        #embeddings_regularizer=regularizers.l2(0.0001),
                        trainable=True)(inp)

        emb = ZeroMaskedLayer()(emb)
        emb = SpatialDropout1D(0.0)(emb)
        if self.bidirectional:
            enc = Bidirectional(CuDNNGRU(int(self.gru_dim), return_sequences=True, return_state=True, 
                                         ))(emb)
            x = enc[0]
            state = enc[1]
        else:
            x, state = CuDNNGRU(int(self.gru_dim), return_sequences=True, return_state=True,
                            #kernel_regularizer=regularizers.l2(0.0001),
                            # recurrent_regularizer=regularizers.l2(0.0001),
                             #bias_regularizer=regularizers.l2(0.0001)
                               )(emb)
            #x = SpatialDropout1D(0.5)(x)
        
        if self.pool_type == 'avg':
            x = GlobalAveragePooling1D()(x)
            x = concatenate([x, state])
            
        elif self.pool_type == 'max':
            x = GlobalMaxPool1D()(x)
            x = concatenate([x, state])
            
        elif self.pool_type == 'attn':
            x = AttentionLayer(MAX_LEN)(x)
            x = concatenate([x, state])
            
        elif self.pool_type == 'all':
            x1 = GlobalAveragePooling1D()(x)
            x2 = GlobalMaxPool1D()(x)
            x3 = AttentionLayer(MAX_LEN)(x)
            x = concatenate([x2, x3, state])
    
        #x = Dropout(0.5)(x)
        x = Dense(self.dense_dim)(x)
        x = PReLU()(x)

        out = Dense(self.out_dim, activation="sigmoid")(x)
        if self.optimizer == 'adam':
            opt = Adam(lr=0.001, decay=0.0, clipnorm=1.0)
        elif self.optimizer == 'rmsprop':
            opt = RMSprop(clipnorm=1.0)
        model = Model(inputs=inp, outputs=out)
        model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
        return model
    
    def fit(self, X, y):
        self.model = self._build_model()
        
        if self.callbacks:
            self.model.fit(X, y, batch_size=self.batch_size, epochs=self.epochs,
                       verbose=self.verbose,
                       callbacks=self.callbacks,
                       shuffle=True)
        else:
            self.model.fit(X, y, batch_size=self.batch_size, epochs=self.epochs,
                       verbose=self.verbose,
                       shuffle=True)
        return self
    
    def predict(self, X, y=None):
        if self.model:
            y_hat = self.model.predict(X, batch_size=2048)
        else:
            raise ValueError("Model not fit yet")
        return y_hat

In [13]:
def lr_decay(epoch):
    if epoch == 0:
        return 0.0009
    if epoch == 1:
        return 0.0007
    if epoch == 2:
        return 0.001
    if epoch == 3:
        return 0.00001


def shuffle_crossvalidator(model, cvlist, X, y):
    y_trues = []
    y_preds = []
    LRDecay = LearningRateScheduler(lr_decay)

    for tr_index, val_index in cvlist2:
        X_tr, y_tr = X[tr_index, :], y[tr_index, :]
        X_val, y_val = X[val_index, :], y[val_index, :]
        RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)

        model.set_params(**{'callbacks':[RocAuc, LRDecay]})
        model.fit(X_tr, y_tr)

        y_pred = model.predict(X_val)
        print("ROC AUC for this fold is ", roc_auc_score(y_val, y_pred))
        y_trues.append(y_val)
        y_preds.append(y_pred)
        K.clear_session()
        #break
    score = roc_auc_score(np.concatenate(y_trues), np.concatenate(y_preds))
    print("Overall score on 10 fold CV is {}".format(score))
    
    return y_preds, y_trues, score

def outoffold_crossvalidator(model_params, cvlist, X, y):
    y_preds = np.zeros(y.shape)
    LRDecay = LearningRateScheduler(lr_decay)

    for tr_index, val_index in cvlist2:
        X_tr, y_tr = X[tr_index, :], y[tr_index, :]
        X_val, y_val = X[val_index, :], y[val_index, :]
        RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)
        
        model.set_params(**{'callbacks':[RocAuc, LRDecay]})
        model.fit(X_tr, y_tr)

        y_pred = model.predict(X_val)
        print("ROC AUC for this fold is ", roc_auc_score(y_val, y_pred))
        y_preds[val_idx] = y_pred
        K.clear_session()
        break
    score = roc_auc_score(y, y_preds)
    print("Overall score on 10 fold CV is {}".format(score))
    
    return y_preds, y_trues, score


In [19]:
model = GRUClassifier(gru_dim=400, dense_dim=600, initial_weights=embedding_matrix, bidirectional=False,
                    batch_size=128, epochs=2, optimizer='adam', pool_type='all')

y_preds, y_trues, _ = shuffle_crossvalidator(model, cvlist2, X_train, y)

Epoch 1/2
151552/151592 [============================>.] - ETA: 0s - loss: 0.0500 - acc: 0.9814
 ROC-AUC - epoch: 1 - score: 0.989340 

151592/151592 [==============================] - 109s 717us/step - loss: 0.0500 - acc: 0.9814
Epoch 2/2
151552/151592 [============================>.] - ETA: 0s - loss: 0.0343 - acc: 0.9860
 ROC-AUC - epoch: 2 - score: 0.989545 

151592/151592 [==============================] - 109s 721us/step - loss: 0.0343 - acc: 0.9860


InternalError: Failed to call ThenRnnForward
	 [[Node: cu_dnngru_1/CudnnRNN = CudnnRNN[T=DT_FLOAT, direction="unidirectional", dropout=0, input_mode="linear_input", is_training=true, rnn_mode="gru", seed=87654321, seed2=0, _device="/job:localhost/replica:0/task:0/device:GPU:0"](cu_dnngru_1/transpose, cu_dnngru_1/ExpandDims_1, cu_dnngru_1/Const_1, cu_dnngru_1/concat)]]
	 [[Node: dense_2/Sigmoid/_183 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_220_dense_2/Sigmoid", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'cu_dnngru_1/CudnnRNN', defined at:
  File "/home/mohsin/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/mohsin/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/mohsin/anac

In [ ]:

#train_gru()
model = GRUClassifier(initial_weights=embedding_matrix, bidirectonal=True,
                    batch_size=128, epochs=2, optimizer='adam')

y_preds, y_trues, _ = shuffle_crossvalidator(model, cvlist2, X_train, y)

In [ ]:
from hyperopt import fmin, tpe, hp


best = fmin(train_gru,
    space=space,
    algo=tpe.suggest,
    max_evals=100)
print(best)

In [ ]:
from hyperopt import hp

def uniform_int(name, lower, upper):
    # `quniform` returns:
    # round(uniform(low, high) / q) * q
    return hp.quniform(name, lower, upper, q=1)

def loguniform_int(name, lower, upper):
    # Do not forget to make a logarithm for the
    # lower and upper bounds.
    return hp.qloguniform(name, np.log(lower), np.log(upper), q=1)

parameter_space = {
    'step': hp.uniform('step', 0.01, 0.5),
    'layers': hp.choice('layers', [{
        'n_layers': 1,
        'n_units_layer': [
            uniform_int('n_units_layer_11', 50, 500),
        ],
    }, {
        'n_layers': 2,
        'n_units_layer': [
            uniform_int('n_units_layer_21', 50, 500),
            uniform_int('n_units_layer_22', 50, 500),
        ],
    }]),
    'act_func_type': hp.choice('act_func_type', [
        layers.Relu,
        layers.PRelu,
        layers.Elu,
        layers.Tanh,
        layers.Sigmoid
    ]),

    'dropout': hp.uniform('dropout', 0, 0.5),
    'batch_size': loguniform_int('batch_size', 16, 512),
}

In [23]:
import gc 
gc.collect()
K.clear_session()

In [27]:
np.concatenate(y_preds).shape
y_trues.shape

NameError: name 'y_trues' is not defined

In [15]:
import skopt
from skopt import gp_minimize, gbrt_minimize
from skopt.space import Real, Integer, Categorical

In [20]:
space = [Integer(16, 256), #name='embed_size'),
         Integer(16, 256),# name='gru_dim'),
         Integer(64, 1024),# name='dense_dim'),
         Real(0.0001, 0.005, "log-uniform"), #name='lr1'),
         Real(1e-5, 0.002, "log-uniform"), #name='lr2'),
         Real(1e-8, 0.001, "log-uniform"), #name='decay'),
         Integer(32, 256),# name=batch'),
        ]

res_gp = gbrt_minimize(train_gru, space, n_calls=100, random_state=0)

(?, 200, 188)
Epoch 1/2
151389/151592 [============================>.] - ETA: 0s - loss: 0.0622 - acc: 0.9795
 ROC-AUC - epoch: 1 - score: 0.982213 

151592/151592 [==============================] - 36s 237us/step - loss: 0.0621 - acc: 0.9795
Epoch 2/2
151389/151592 [============================>.] - ETA: 0s - loss: 0.0363 - acc: 0.9857
 ROC-AUC - epoch: 2 - score: 0.983151 

151592/151592 [==============================] - 36s 238us/step - loss: 0.0364 - acc: 0.9857
ROC AUC for this fold is  0.9831507887571517
(?, 200, 188)
Epoch 1/2
151389/151592 [============================>.] - ETA: 0s - loss: 0.0624 - acc: 0.9793
 ROC-AUC - epoch: 1 - score: 0.982551 

151592/151592 [==============================] - 36s 234us/step - loss: 0.0624 - acc: 0.9793
Epoch 2/2
151389/151592 [============================>.] - ETA: 0s - loss: 0.0368 - acc: 0.9855
 ROC-AUC - epoch: 2 - score: 0.984904 

151592/151592 [==============================] - 36s 234us/step - loss: 0.0368 - acc: 0.9855
ROC AUC for

KeyboardInterrupt: 

In [129]:
y_trues

NameError: name 'y_trues' is not defined

In [43]:
roc_auc_score(y, y_preds)

0.9861737784746518

In [41]:
model = get_model()
model.fit(X_t, y, batch_size=32, epochs=3, validation_split=0.0, verbose=1, 
              callbacks=[RocAuc, LRDecay])
y_test_preds = model.predict([X_te], batch_size=1024, verbose=1)


Epoch 1/3
159456/159571 [============================>.] - ETA: 0s - loss: 0.0498 - acc: 0.9818
 ROC-AUC - epoch: 1 - score: 0.992135 

159571/159571 [==============================] - 84s 528us/step - loss: 0.0498 - acc: 0.9818
Epoch 2/3
159488/159571 [============================>.] - ETA: 0s - loss: 0.0384 - acc: 0.9850
 ROC-AUC - epoch: 2 - score: 0.995249 

159571/159571 [==============================] - 83s 523us/step - loss: 0.0384 - acc: 0.9850
Epoch 3/3
159488/159571 [============================>.] - ETA: 0s - loss: 0.0307 - acc: 0.9880
 ROC-AUC - epoch: 3 - score: 0.995429 

153164/153164 [==============================] - 2s 15us/step


In [25]:
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
y_trues = train[label_cols].values
y_preds2 = np.zeros((X_t.shape[0], len(label_cols)))
y_test_preds2 = np.zeros((X_te.shape[0], len(label_cols)))
for i, col in enumerate(label_cols):
    y = y_trues[:, i]
    #model = RandomForestClassifier(n_estimators=100, max_depth=6, min_samples_leaf=50, class_weight='balanced', n_jobs=-1)
    model = lgb.LGBMClassifier(n_estimators=100, num_leaves=5, learning_rate=0.03, 
                               subsample=0.9, colsample_bytree=0.9)
    y_preds2[:, i] = cross_val_predict(model, y_preds, y, cv=cvlist, n_jobs=1, method='predict_proba')[:,1]
    y_test_preds2[:, i] = model.fit(y_preds, y).predict_proba(y_test_preds)[:,1]
    print("Score for class {} is {}".format(col, roc_auc_score(y, y_preds2[:, i])))
print("Over auc score", roc_auc_score(y_trues, y_preds2))

Score for class toxic is 0.9791088111561013
Score for class severe_toxic is 0.9896361881404786
Score for class obscene is 0.989792727484462
Score for class threat is 0.9834375512104746
Score for class insult is 0.9851961777705942
Score for class identity_hate is 0.9837426177272286
Over auc score 0.9851523455815565


In [42]:
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[label_cols] = y_test_preds
sample_submission.to_csv('nn_submission.csv', index=False)

In [ ]:
sample_submission[label_cols] = y_test_preds2
sample_submission.to_csv('nn_lgbmeta_submission.csv', index=False)